In [48]:
import cv2
import os
import subprocess
import numpy as np
import pandas as pd
import time
import json
#see what we need from these 
import pandas as pd
import numpy as np
import re
import urllib.parse

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.metrics import accuracy_score 

import matplotlib.pyplot as plt
from sklearn import tree
from six import StringIO  
from IPython.display import Image  
import pydotplus
from matplotlib.colors import LinearSegmentedColormap
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# import the regressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [49]:

#merge tables with features and table with labels
#names should be same

#we have a path to the main folder 

def mergeFeaturesAndLabels(folder):
    
    """
    Merges the features and labels from CSV files in the specified folder and saves the result in a new CSV file.

    Parameters:
        folder (str): The path to the folder containing 'features.csv' and 'labels.csv' files.

    Returns:
        Merged DataFrame.
    """

    path_to_features = f'{folder}/features.csv'
    path_to_labels = f'{folder}/labels.csv'

    #upload 
    df_features = pd.read_csv(path_to_features)
    df_labels = pd.read_csv(path_to_labels)

    #merge tables 
    df = pd.merge(df_features, df_labels, on=['Name'])

    #raname tables if needed
    df.drop('Unnamed: 0_x', axis=1, inplace=True)
    df.drop('Unnamed: 0_y', axis=1, inplace=True)
    df.rename(columns={'Bitrate_x': 'Bitrate'}, inplace=True)
    df.rename(columns={'Bitrate_y': 'Label'}, inplace=True)

    #delete rows where label -1
    df = df[df['Label'] != -1]

    #save file
    # Specify the file path and name for the CSV file
    csv_file_path = f'{folder}/features_and_labels.csv'

    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    return df

In [86]:
#DECISION TREE
#folder with features and labels
#initiates def mergeFeaturesAndLabels

def runDecisionTreeRegressor(folder):
    """
    Runs a Decision Tree Regressor on the data in the specified folder, predicts the labels, and calculates the Mean Squared Error (MSE).

    Parameters:
        folder (str): The path to the folder containing 'features.csv' and 'labels.csv' files.

    Returns:
        regressor: The trained Decision Tree Regressor object.
    """
    mergeFeaturesAndLabels(folder)
    
    df = pd.read_csv(f"{folder}/features_and_labels.csv")
    # df is a table with features and labels 
    #where label is in the last column called Label
    #Name is the first column called Name
    #THER COLUMNS ARE FEATURES

    #names of the columns with features, 
    #can also just take all columns apart from first with name and last with label
    
    df_copy= df.copy()
    df_copy.drop(columns=["Name", "Label"], inplace=True)
    list_of_features = df_copy.columns.tolist()
    X = df[list_of_features] #Features
    y = df.Label #Labels

    #split the data into train and test 

    #Split the data into training set and test set
    # 67% training and 33% test
    
    #random state can be removed if we dont want to see the same split each time
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 
    # create a regressor object
    regressor = DecisionTreeRegressor(random_state = 0) 

    # fit the regressor with X and Y data
    regressor.fit(X_train, y_train)

    # Predict
    y_predicted = regressor.predict(X_test)

    # Calculate the MSE
    mse = mean_squared_error(y_test, y_predicted)
    print("Mean Squared Error:", mse)


    # Get the feature importances
    feature_importances = regressor.feature_importances_

    # Print the feature importances
    print("Features importance \n")
    for item1, item2 in zip(list_of_features, feature_importances):
        print(f"{item1}: {item2}")
    

    test_table = X_test.copy() 
    test_table['Name'] = df['Name'].iloc[X_test.index]
    test_table['Label'] = y_test
    test_table['Prediction'] = y_predicted
    #save file
    # Specify the file path and name for the CSV file
    csv_file_path = "predictions.csv"

    # Save the DataFrame to a CSV file
    test_table.to_csv(csv_file_path, index=False)

    return regressor

In [87]:
#test

folder = "/Users/anastasiya/Documents/MASTER/SOSE23/Deep_Encode/test"
regressor = runDecisionTreeRegressor(folder)


Mean Squared Error: 167.09838998211092
Features importance 

Bitrate: 0.3723335834337365
width: 0.0002473935296013186
height: 5.942048719124421e-05
spatial_info: 0.32772797260706965
temporal_info: 0.11060316796667803
fps: 0.003380344057584143
entropy: 0.18564811791813896
